# Solar Energy Forecasting

## Introduction

Some exploratory data analysis to better understand the data

## Historical Data (Radiation, Air temperature, Wind speed)
Weather data for location: Freiburg im Breisgau  
Using data from the year 2020 (because it is the most recent year there is data for on the website)  
Get historical weather data from: https://joint-research-centre.ec.europa.eu/photovoltaic-geographical-information-system-pvgis/pvgis-tools/hourly-radiation_en

## Historical Data (Solar energy production)
Energy data for location: Transnet (Baden-Württemberg)  
Using data from the year 2020  
Get historical energy production data from: https://www.smard.de (https://www.smard.de/en/downloadcenter/download-market-data/?downloadAttributes=%7B%22selectedCategory%22:1,%22selectedSubCategory%22:1,%22selectedRegion%22:%22TransnetBW%22,%22selectedFileType%22:%22CSV%22,%22from%22:1577833200000,%22to%22:1609541999999%7D)

## Weather forecast data (Radiation, Air temperature, Wind speed)
Get weather forecast from: https://www.weatherapi.com/my/ 14 days is for free  
Do prediction, then we need to wait and download again from smard.de and we can then compare our prediction to the truth



In [3]:
import pandas as pd
import pickle

# Load the pickle file
with open("solar_data.pkl", "rb") as file:
    df = pickle.load(file)

print(df.head())
print(df.describe())

            time  G(i)  H_sun   T2m  WS10m
0  20050101:0011   0.0    0.0  6.83   3.59
1  20050101:0111   0.0    0.0  6.80   3.45
2  20050101:0211   0.0    0.0  6.79   3.24
3  20050101:0311   0.0    0.0  6.44   2.83
4  20050101:0411   0.0    0.0  6.26   2.62
               G(i)         H_sun           T2m         WS10m
count  17520.000000  17520.000000  17520.000000  17520.000000
mean     126.499125     12.472856      9.985207      2.917154
std      206.174483     17.108247      8.817461      1.315834
min        0.000000     -0.610000    -21.620000      0.000000
25%        0.000000      0.000000      3.040000      2.000000
50%        0.000000      0.000000     10.510000      2.690000
75%      178.002500     22.010000     16.480000      3.660000
max      912.010000     60.910000     35.340000      9.520000
